## Import and format json data
Importing playlists from jsons and formatting them into ratings data frame. (Get data from https://owncloud.tuwien.ac.at/index.php/s/A8Wx2TpFr0WznZh)

In [156]:
import pandas as pd
import numpy as np
import json

# load data
train = json.load(open('data/train.json', encoding = "utf8"))

dev = json.load(open('data/dev.json', encoding = "utf8"))
dev_trun = json.load(open('data/dev_trun.json', encoding="utf8"))

test = json.load(open('data/test.json', encoding = "utf8"))
test_trun = json.load(open('data/test_trun.json', encoding = "utf8"))

# create ratings df with training + dev for validation (equivalent to 'ratings_train' data frame in Assignment 4.1)
rating_list = []
only_dev_list = []

seen = set()
track_list = []
for playlist in train['playlists']:
    for track in playlist['tracks']:
        rating_list.append([playlist['pid'], playlist['name'], track['track_uri'], 1.0])
        if track['track_uri'] not in seen:
            seen.add(track['track_uri'])
            track_list.append([track['track_uri'], track['track_name'], track['artist_uri'], track['artist_name'], track['album_uri'], track['album_name'], track['duration_ms']])

for playlist in dev_trun['playlists']:
    for track in playlist['tracks']:
        rating_list.append([playlist['pid'], playlist['name'], track['track_uri'], 1.0])
        only_dev_list.append([playlist['pid'], playlist['name'], track['track_uri'], 1.0])
        if track['track_uri'] not in seen:
            seen.add(track['track_uri'])
            track_list.append([track['track_uri'], track['track_name'], track['artist_uri'], track['artist_name'], track['album_uri'], track['album_name'], track['duration_ms']])

ratings_train_dev = pd.DataFrame(rating_list, columns = ['pid', 'name', 'track_uri', 'rating'])
ratings_only_dev = pd.DataFrame(only_dev_list, columns = ['pid', 'name', 'track_uri', 'rating'])
tracks = pd.DataFrame(track_list, columns = ['track_uri', 'track_name', 'artist_uri', 'artist_name', 'album_uri', 'album_name', 'duration_ms'])
                           
# create ratings df for testing
rating_list = []
for playlist in train['playlists']:
    for track in playlist['tracks']:
        rating_list.append([playlist['pid'], playlist['name'], track['track_uri'], 1.0])
for playlist in test_trun['playlists']:
    for track in playlist['tracks']:
        rating_list.append([playlist['pid'], playlist['name'], track['track_uri'], 1.0])

ratings_train_test = pd.DataFrame(rating_list, columns = ['pid', 'name', 'track_uri', 'rating'])

# holdouts ('topN' equivalent)
dev_holdouts = {playlist['pid']:playlist['num_holdouts'] for playlist in dev['playlists']}
test_holdouts = {playlist['pid']:playlist['num_holdouts'] for playlist in test['playlists']}

#ratings_train_dev.head()
tracks.head()

# not sure if train and dev need to be combined.. train alone doesn't miss any tracks, 
# so we cannot test the model on it. Combining it with def, we can make predictions on
# missing tracks in dev playlists. EDIT: I think combining is right. Also combining train + test. 
# Otherwise, train isn't part of the model.


,track_uri,track_name,artist_uri,artist_name,album_uri,album_name,duration_ms
0,spotify:track:2CFGQXM3exBO8U9WGve2GY,Elle me dit,spotify:artist:5MmVJVhhYKQ86izuGHzJYA,MIKA,spotify:album:0zY3JRYHC0rycbFUqGcRpV,The Origin Of Love,216933
1,spotify:track:09TcIuH1ZO7i4vicWKoaN2,Papaoutai,spotify:artist:5j4HeCoUlzhfWtjAfM1acR,Stromae,spotify:album:4WW3ME0M2dUoAK5hmbXsN7,racine carrée,232146
2,spotify:track:3Uyt0WO3wOopnUBCe9BaXl,Sur ma route,spotify:artist:7lMgpN1tEBQKpRoUMKB8iw,Black M,spotify:album:1D2Rs9qcENebbiDR5wk88T,Les yeux plus gros que le monde,252573
3,spotify:track:4h95OrjKWYzVIai1EkvZbt,Ta fête,spotify:artist:5j4HeCoUlzhfWtjAfM1acR,Stromae,spotify:album:4WW3ME0M2dUoAK5hmbXsN7,racine carrée,175426
4,spotify:track:3pCN6X0566rmU6P5ZgFIYu,bâtard,spotify:artist:5j4HeCoUlzhfWtjAfM1acR,Stromae,spotify:album:4WW3ME0M2dUoAK5hmbXsN7,racine carrée,208826


## Import and test recommenders

In [145]:
%load_ext autoreload
%autoreload 2
from Recommender_CB import ContentBasedRecommender
from Recommender_CF_UU import UUCFRecommender
from Recommender_MF import MFRecommender

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [146]:
## Matrix Fac.
mfr = MFRecommender()
mfr.build_model(ratings_train_dev)

recs = mfr.recommend('498917', topN = holdouts['498917'])
print(recs)


## UU-CB
uucf = UUCFRecommender()
uucf.build_model(ratings_train_dev)

recs = uucf.recommend('498917', topN = holdouts['498917'])
print(recs)

AttributeError: 'DataFrame' object has no attribute 'movieId'

In [166]:
## Content-based 
br = ContentBasedRecommender('extended')
cbr.build_model(ratings_only_dev, tracks)
#br.build_model(ratings_train_dev, tracks)

ok
ok playlist profiles


In [167]:
cb_recs = cbr.recommend(498917, topN = dev_holdouts.get(498917))
print(cb_recs)

['spotify:track:30lgD1UuHczwlxa7NZFeSQ', 'spotify:track:0pBLfQ5JBjh12H6DGZjMwM', 'spotify:track:7pJgjBf82BrUQ3z7HdQvW1', 'spotify:track:29DwxnpFOLlm1fKZOitwAA', 'spotify:track:3CJvmtWw2bJsudbAC5uCQk', 'spotify:track:3viZcRlE38APncWJ5VGM4q', 'spotify:track:3IMXLQk09YmvNb2tdT4aWk', 'spotify:track:5U2m76qAMdr3gXI0g6ptlG', 'spotify:track:3LuJPqTyPuVfm9VemXTWHo', 'spotify:track:6ZANrVuAMp2rpjhfbOuJly', 'spotify:track:37IToWWYUc9nuH3ijz4tqV', 'spotify:track:4dzOxPZFYX77xpMwv4EhnO', 'spotify:track:1lbWbnWiEbAya5FlCzfsrq', 'spotify:track:4hAhyE1iL07EIUJnqw2bsV', 'spotify:track:57nNNkgk768QVXq3uHxu5e', 'spotify:track:21zv5Hi0SQSvLyaKPiKreg', 'spotify:track:2E26jtkc5BONr50yOwyupV', 'spotify:track:5P3yUXUC9rZPJPNmYGKEAz', 'spotify:track:3CRDbSIZ4r5MsZ0YwxuEkn', 'spotify:track:4Oyl6oYSNeeZZP0OAxPVaU', 'spotify:track:22OqansuvEMQu9W0EfiGTI', 'spotify:track:40sRwFjfrRnaTdYPJIg4CP', 'spotify:track:5cbpoIu3YjoOwbBDGUEp3P', 'spotify:track:4WUduEPZZG6wiE9B5IklnY', 'spotify:track:57yL3161hUMuw06zzzUCHi',

## Parameter tuning
Parameters are tuned through evaluating performance of different parameter settings on the train+dev (?) set. Those settings will be used for the test set.

In [175]:
import metrics
infile = 'data/dev_gt.txt'
split_pits = None
with open(infile, 'rt') as f_i:
    split_pits = [z.strip(' ()') for z in f_i.read().strip().split('\n')]
if split_pits:
    target_sets = set(tuple(z.strip()
                 for z in tup.split(', ')) for tup in split_pits)

target_set = set() #TODO: get with pid

#TODO: Evaluate results:
print(metrics.r_precision(target_set, cb_recs, dev_holdouts.get(498917)))
    
#TODO: Parameter tuning
## Content-based 
cbr = ContentBasedRecommender('extended')

('498797', 'spotify:track:2cSXotAohYx0EWCNljCzs7', 'spotify:track:0lPXFrKyl8HkdQ2E6NmNja', 'spotify:track:0dReCfBuhZmrqMv4WBc8od', 'spotify:track:5828oiHxe6DSSAuIrPSeVT', 'spotify:track:6dhnmkbc9Z7pGsUyUgZKGX', 'spotify:track:62qI8m0sXpzlSjYYx0nZ83', 'spotify:track:2DAsLftcRKP3iarCPmI1RY', 'spotify:track:6UwB2R8I5LoUmmYc0mphdl', 'spotify:track:3C9fAn3IgajBTwR274huch', 'spotify:track:79gkHjNTSyGPXEyGkNQ8zH', 'spotify:track:0gdXjabcDhUeitLTzxxPFd', 'spotify:track:0qO4GfXADY9QfgcgtrasT9', 'spotify:track:0IDyf9eri6YrUs4yeQfCZZ', 'spotify:track:1EZypwk0xcj64ZLAglhLs2', 'spotify:track:7CX6W4Mf98wMgX0izqRpAl', 'spotify:track:22seP38foydqUMp5Fvkcjm', 'spotify:track:7Fk4GdWusuxSDabJKzyzbt', 'spotify:track:5dMzxEVxEWRJysu2dOYEzR', 'spotify:track:68RvCYDKtz9HR1WMR9D9VV', 'spotify:track:5Z8lZ3FhHDMF85kZXXGhFZ', 'spotify:track:4B1sb2YVO03sBmyR8bvzEB', 'spotify:track:4AQZNgdG73f2ZdC6VUZqBL', 'spotify:track:5QkhJ1APhxew8pYaj1TXaN', 'spotify:track:52FjcsfkCImlf6HSN83udR', 'spotify:track:15Hx1oi6C3AxYh

## Evaluation on test set 

In [163]:
import metrics
infile = 'data/test_gt.txt'
split_pits = None
with open(infile, 'rt') as f_i:
    split_pits = [z.strip(' ()') for z in f_i.read().strip().split('\n')]
if split_pits:
    target_set = set(tuple(z.strip()
                 for z in tup.split(', ')) for tup in split_pits)



#TODO: evaluation on test set


# Content-based
#cbr.build_model(ratings_train_test, tracks)
#recs = cbr.recommend(498917, topN = test_holdouts.get(498917))
metrics.r_precision(target_set, recs)

0.0